In [23]:
from random import randint

In [24]:
def random_name_from_file(file_path) -> str:
    with open(file_path, "r") as file:
        names = file.readlines()
        return names[randint(0, len(names) - 1)].strip()

In [25]:
    
base_url = "https://docs.google.com/forms/d/e/1FAIpQLScoJzoGy9gIiQ3nU-JDKS7XqnbMnETmY9F2oUYPA7u3NUZfpA/viewform?usp=pp_url"

entry_list = ["2092238618", "215193630", "588393791", "1753222212", "572352119"]

multi_choice_questions = ["588393791"]

answers_1753222212 = ["0 - 7 days", "7 - 14 days", "14 - 30 days", "longer than 30 days"]
answers_572352119 = ["Yes", "No"]
answers_588393791 = ["Churches", "Landmarks","Monuments","Castles","Statues and sculptures","Sports facilities","Cinema", "Caves", "Museums", "Parks", "Theatres", "Zoos"]

def random_set_from_list(lst: list, n: int) -> list:
    return [lst[randint(0, len(lst) - 1)] for i in range(n)]

possible_answers = {
    "1753222212": answers_1753222212,
    "572352119": answers_572352119,
    "588393791": answers_588393791
}


In [26]:
def generate_url(entry_list, multi_choice_questions, possible_answers):
    url = base_url
    name = random_name_from_file("names.txt") + " " + random_name_from_file("surnames.txt")


    for entry in entry_list:
        
        if entry in multi_choice_questions:
            # url += possible_answers[entry][randint(0, len(possible_answers[entry]) - 1)]
            set_size = randint(1, len(possible_answers[entry]))
            answ = random_set_from_list(possible_answers[entry],set_size)
            for a in answ:
                url += f"&entry.{entry}="
                url += a
        elif entry == "2092238618":
            url += f"&entry.{entry}="
            url += name
        elif entry == "215193630":

            # email
            url += f"&entry.{entry}="
            url += f"{name.lower().replace(' ', '.')}@gmail.com"
        else:
            # use possible answers dict
            url += f"&entry.{entry}="
            url += possible_answers[entry][randint(0, len(possible_answers[entry]) - 1)]


    # urlencode 
    url = url.replace(" ", "+")
    return url

In [27]:
urls = [generate_url(entry_list, multi_choice_questions, possible_answers) for i in range(100)]
for url in urls:
    print(url)
    print("\n")

https://docs.google.com/forms/d/e/1FAIpQLScoJzoGy9gIiQ3nU-JDKS7XqnbMnETmY9F2oUYPA7u3NUZfpA/viewform?usp=pp_url&entry.2092238618=Gabriele+Magnifico&entry.215193630=gabriele.magnifico@gmail.com&entry.588393791=Landmarks&entry.588393791=Zoos&entry.588393791=Theatres&entry.588393791=Museums&entry.588393791=Churches&entry.1753222212=14+-+30+days&entry.572352119=No


https://docs.google.com/forms/d/e/1FAIpQLScoJzoGy9gIiQ3nU-JDKS7XqnbMnETmY9F2oUYPA7u3NUZfpA/viewform?usp=pp_url&entry.2092238618=Suanne+Dittman&entry.215193630=suanne.dittman@gmail.com&entry.588393791=Statues+and+sculptures&entry.588393791=Cinema&entry.588393791=Theatres&entry.588393791=Castles&entry.588393791=Cinema&entry.1753222212=longer+than+30+days&entry.572352119=No


https://docs.google.com/forms/d/e/1FAIpQLScoJzoGy9gIiQ3nU-JDKS7XqnbMnETmY9F2oUYPA7u3NUZfpA/viewform?usp=pp_url&entry.2092238618=Kerri+Peppel&entry.215193630=kerri.peppel@gmail.com&entry.588393791=Zoos&entry.588393791=Museums&entry.588393791=Churches&entry.5883

In [28]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [29]:
driver = webdriver.Firefox()
# button class l4V7wb Fxmcue 
for url in urls:
    driver.get(url)
    button = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, '//div/span/span[text()="Prześlij"]')))
    button.click()
driver.close()